# 201710864 박형철

# 문제 1.

In [15]:
%%writefile src/firstQuestion.js

var Web3 = require('web3');
var web3 = new Web3("http://localhost:8345");
var balance;
async function getInfo() {
    accounts = await web3.eth.getAccounts();
    account0 = accounts[0];
    account1 = accounts[1];
    console.log("ac0:",account0, "ac1:",account1)
    
    balance0 = await web3.eth.getBalance(account0);
    balance1 = await web3.eth.getBalance(account1);
    console.log("bal of ac0:",balance0, "ac1:",balance1);


    await web3.eth.personal.sendTransaction({from:account0, to:account1, value:"1111"}).then(function(err, Resolved) {
        console.log("callback-sending ac0 -> ac1");});
    
    tx = await web3.eth.getTransaction("0x9baccb0e64f1b6ae9b05610ae83f535a128511e6907176f1f86ae3d2ff059ebf");
    console.log("transactionHash:",tx.hash);
    
    gasPrice = tx.gasPrice;
    gasCost = tx.gas;
    
    totalGas = gasPrice*gasCost;
    console.log("gas costs:",totalGas);
    
    console.log("nonce:",tx.nonce);
    
    balance0_ = await web3.eth.getBalance(account0);
    balance1_ = await web3.eth.getBalance(account1);
    
    diff1 = parseInt(balance0_) - parseInt(balance0);
    diff2 = parseInt(balance1_) - parseInt(balance1);
    console.log("bal diff1:",diff1, "bal diff2:",diff2);
    
    var sum=0;
    for (var i=0; i<10; i++) {
        sum = sum + parseInt(await web3.eth.getBalance(accounts[i]));
        console.log("sum:",i,"adding",i,"bal:",sum);
    }
    
    console.log("balance total:",sum);
         
}
getInfo()



Overwriting src/firstQuestion.js


In [16]:
!node src/firstQuestion.js

ac0: 0xF8cB02C734e1197F8a5E8604c8a022B2dbD2113C ac1: 0x6A12ffA57FE1c654821F22873e66b721a83c40EB
bal of ac0: 999999938578803789667 ac1: 1000000000000000003333
callback-sending ac0 -> ac1
/Users/broiron/Code/201710864/src/firstQuestion.js:20
    console.log("transactionHash:",tx.hash);
                                      ^

TypeError: Cannot read properties of null (reading 'hash')
    at getInfo (/Users/broiron/Code/201710864/src/firstQuestion.js:20:39)
    at processTicksAndRejections (node:internal/process/task_queues:96:5)

Node.js v17.8.0


# 문제 2. '풀지못함'

[vm]from: 0x5B3...eddC4to: Counter.(constructor)value: 0 weidata: 0x608...c0033logs: 0hash: 0x636...65284
Debug
status	true Transaction mined and execution succeed
transaction hash	0x6361c05061a00b74f38213555456aadbeb1bda418ebbd7f0e600d77441965284
from	0x5B38Da6a701c568545dCfcB03FcB875f56beddC4
to	Counter.(constructor)
gas	193949 gas
transaction cost	168651 gas 
execution cost	168651 gas 
input	0x608...c0033
decoded input	{}
decoded output	 - 
logs	[]
val	0 wei

In [75]:
%%writefile src/Counter.sol
pragma solidity ^0.8.0;
contract Counter {
    uint256 counter = 0;
    uint256 startTime;
    function add() public {
        counter++;
    }
    function subtract() public {
        counter--;
    }
    function getCounter() public view returns (uint256) {
        return counter;
    }

    function setTimeCalled() public {
        startTime=block.timestamp;
    }

    function getTimeCalled() view public returns(uint256) {
        return block.timestamp;
    }

}

Writing src/Counter.sol


In [145]:
!echo "var _compiled=`solc --optimize --combined-json abi,bin,interface src/Counter.sol`" > src/CounterGeth.js

Invalid option to --combined-json: interface


In [146]:
!echo "var _compiled=`solc --optimize --combined-json abi,bin src/Counter.sol`" > src/CounterGeth.js

--> src/Counter.sol



In [147]:
!geth --exec 'loadScript("src/CounterGeth.js")' attach http://localhost:8445

undefined


In [148]:
%%writefile src/CounterDeployGeth.js
var primary = eth.accounts[0];
console.log("primary ac: ",primary);
console.log("balance: ",eth.getBalance(primary));
loadScript('src/CounterGeth.js');
contractName=Object.keys(_compiled.contracts); // 
//_abi=JSON.parse(_compiled.contracts['src/Example.sol:Example'].abi)
//_bin=_compiled.contracts['src/Example.sol:Example'].bin
_abi=JSON.parse(_compiled.contracts[contractName[0]].abi)
_bin=_compiled.contracts[contractName[0]].bin

_class=eth.contract(_abi);
console.log('bin code: ', _bin)
//this async way does not work from the Jupyter Notebook
_object=_class.new({from:primary,data:'0x'+_bin,gas:1000000}, function(err, contract) {
  if (!err && contract.address)t
    console.log("contractAddress: ", contract.address);
    console.log("transactionHash: ", contract.transactionHash);
});

Overwriting src/CounterDeployGeth.js


In [149]:
!geth --exec "loadScript('src/CounterDeployGeth.js')" attach http://localhost:8445

primary ac:  0xbc3995a91ef543781cd777de3b192acbc03a74a0
balance:  114687520999999900000
GoError: Error while compiling or running script: SyntaxError: invalid character 'o' looking for beginning of value at parse (native)
	at github.com/ethereum/go-ethereum/internal/jsre.MakeCallback.func1 (native)
	at <eval>:1:12(1)



In [ ]:
!geth --exec "eth.getTransactionReceipt('')" attach http://localhost:8445

In [144]:
%%writefile src/CounterUseGeth.js
loadScript('src/TCounterGeth.js')
contractName=Object.keys(_compiled.contracts)
_abi=JSON.parse(_compiled.contracts[contractName[0]].abi)
//_abi=JSON.parse(_compiled.contracts['src/Example.sol:Example'].abi)
var _contract=eth.contract(_abi);
var _address="";
var _instance=eth.contract(_abi).at(_address);
console.log(_instance.getCounter.call());
_instance.setTimeCalled.sendTransaction({from:eth.accounts[0]});
console.log(_instance.getTimeCalled.call());
_instance.add.sendTransaction({from:eth.accounts[0]});

console.log(_instance.getCounter.call());
console.log(_instance.getTimeCalled.call());

_instance.add.sendTransaction({from:eth.accounts[0]});

console.log(_instance.getCounter.call());
console.log(_instance.getTimeCalled.call());

Writing src/CounterUseGeth.js
